## Task 1: Introduction

In [1]:
%matplotlib inline
%load_ext tensorboard

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import shutil

## Task 2: Create Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
colors = {
    0: 'red',
    1: 'green',
}

def create_example(x, y):
    c = np.random.randint(0, 2)
    image = 0.5 * np.random.rand(28, 28, 3)
    image[:, :, c] += 0.5 * x / 255.
    return image, y, c

In [ ]:
index = np.random.randint(0, len(x_train))
x, y, c = create_example(x_train[index], y_train[index])

print(f'digit: {y}, color: {colors[c]}')

plt.imshow(x)
plt.show()

## Task 3: Dataset Generator

In [2]:
def generate_data(x, y, batch_size=32):
    num_examples = len(y)
    
    while True:
        x_batch = np.zeros((batch_size, 28, 28, 3))
        y_batch = np.zeros((batch_size, ))
        c_batch = np.zeros((batch_size, ))

        for i in range(0, batch_size):
            index = np.random.randint(0, num_examples)
            image, digit, color = create_example(x[index], y[index])
            x_batch[i] = image
            y_batch[i] = digit
            c_batch[i] = color

        yield x_batch, [y_batch, c_batch]

## Task 4: Create Model

In [2]:
num_filters = 32

input_ = tf.keras.layers.Input(shape=(28, 28, 3), name='input')

input_ = Input(MAXLEN_NAME_CHAR_LSTM)


conv_1 = tf.keras.layers.Conv2D(num_filters, 3, name='conv_1')(input_)
act_1 = tf.keras.layers.Activation('relu', name='act_1')(conv_1)

pool_1 = tf.keras.layers.MaxPool2D(4, name='pool_1')(act_1)
flat_1 = tf.keras.layers.Flatten(name='flat_1')(pool_1)

conv_2 = tf.keras.layers.Conv2D(num_filters, 3, padding='same', name='conv_2')(act_1)
act_2 = tf.keras.layers.Activation('relu', name='act_2')(conv_2)

conv_3 = tf.keras.layers.Conv2D(num_filters, 3, padding='same', name='conv_3')(act_2)
add = tf.keras.layers.Add(name='add')([act_1, conv_3])

act_3 = tf.keras.layers.Activation('relu', name='act_3')(add)
pool_2 = tf.keras.layers.MaxPool2D(4, name='pool_2')(act_3)

flat_2 = tf.keras.layers.Flatten(name='flat_2')(pool_2)

digit = tf.keras.layers.Dense(10, activation='softmax', name='digit')(flat_2)
color = tf.keras.layers.Dense(1, activation='sigmoid', name='color')(flat_1)

model = tf.keras.models.Model(input_, [digit, color])

model.compile(
    loss={
        'digit': 'sparse_categorical_crossentropy',
        'color': 'binary_crossentropy'
    },
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 28, 28, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 26, 26, 32)   896         input[0][0]                      
__________________________________________________________________________________________________
act_1 (Activation)              (None, 26, 26, 32)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 26, 26, 32)   9248        act_1[0][0]                      
______________________________________________________________________________________________

2021-11-08 10:28:11.662272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64
2021-11-08 10:28:11.663181: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-08 10:28:11.663517: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow w

In [3]:
!pip install pydot
!pip install graphviz
!pip install pydotplus

In [9]:
tf.keras.utils.plot_model(model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Task 5: Training the Model

In [ ]:
class Logger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        digit_accuracy = logs.get('digit_accuracy')
        color_accuracy = logs.get('color_accuracy')
        val_digit_accuracy = logs.get('val_digit_accuracy')
        val_color_accuracy = logs.get('val_color_accuracy')
        print('='*30, epoch + 1, '='*30)
        print(f'digit_accuracy: {digit_accuracy:.2f}, color_accuracy: {color_accuracy:.2f}')
        print(f'val_digit_accuracy: {val_digit_accuracy:.2f}, val_color_accuracy: {val_color_accuracy:.2f}')

In [ ]:
train_gen = generate_data(x_train, y_train)
val_gen = generate_data(x_test, y_test)

_ = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=200,
    validation_steps=100,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        Logger()
    ],
    verbose=False
)

In [ ]:
%tensorboard --logdir logs

## Task 6: Final Predictions

In [ ]:
def test_model(show=True):
    x, [y, c] = next(test)
    
    preds = model.predict(x)
    pred_digit = np.argmax(preds[0])
    pred_color = int(preds[1] > 0.5)
    gt_digit = int(y[0])
    gt_color = int(c[0])
    
    plt.imshow(x[0])
    if show:
        print(f'GT: {gt_digit}, {colors[gt_color]}')
        print(f'Pr: {pred_digit}, {colors[pred_color]}')
        plt.show()
    else:
        col = 'green' if gt_digit == pred_digit and gt_color == pred_color else 'red'
        plt.ylabel(f'GT: {gt_digit}, {colors[gt_color]}', color=col)
        plt.xlabel(f'Pr: {pred_digit}, {colors[pred_color]}', color=col)
        plt.xticks([])
        plt.yticks([])


test = generate_data(x_test, y_test, batch_size=1)

In [ ]:
test_model()

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 16):
    plt.subplot(4, 4, i + 1)
    test_model(False)
plt.show()